## Draft

In [ ]:

# visualize.plot_boxplot(
#     df, 
#     'target', 
#     title='Target boxplot', 
#     numeric_label='price, $', 
#     log_scale=True    
# );

In [ ]:
res = df['homeFacts'][1].replace('\'', '"').replace('None', '""')
res[285:290]

'l": "'

In [ ]:
res

'{"atAGlanceFacts": [{"factValue": "2019", "factLabel": "Year built"}, {"factValue": "", "factLabel": "Remodeled year"}, {"factValue": "", "factLabel": "Heating"}, {"factValue": "", "factLabel": "Cooling"}, {"factValue": "", "factLabel": "Parking"}, {"factValue": "5828 sqft", "factLabel": "lotsize"}, {"factValue": "$159/sqft", "factLabel": "Price/sqft"}]}'

In [ ]:
import json
facts_dt = json.loads(res)
facts_dt

{'atAGlanceFacts': [{'factValue': '2019', 'factLabel': 'Year built'},
  {'factValue': '', 'factLabel': 'Remodeled year'},
  {'factValue': '', 'factLabel': 'Heating'},
  {'factValue': '', 'factLabel': 'Cooling'},
  {'factValue': '', 'factLabel': 'Parking'},
  {'factValue': '5828 sqft', 'factLabel': 'lotsize'},
  {'factValue': '$159/sqft', 'factLabel': 'Price/sqft'}]}

In [ ]:
facts_dt['atAGlanceFacts']

[{'factValue': '2019', 'factLabel': 'Year built'},
 {'factValue': '', 'factLabel': 'Remodeled year'},
 {'factValue': '', 'factLabel': 'Heating'},
 {'factValue': '', 'factLabel': 'Cooling'},
 {'factValue': '', 'factLabel': 'Parking'},
 {'factValue': '5828 sqft', 'factLabel': 'lotsize'},
 {'factValue': '$159/sqft', 'factLabel': 'Price/sqft'}]